In [1]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import requests

In [2]:
server = "https://geo.api.vlaanderen.be/BWD-IdentifiedBathingWaters/ogc/features/collections/BWDBW/items?f=application%2Fgeo%2Bjson"

In [3]:
# Azure App Gateway blocks Python requests lib by default
response = requests.get(server, headers={
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
})


In [4]:
data = [item['properties'] for item in response.json()['features']]
data

KeyError: 'features'

In [ ]:
encoding = data[0]['COORDSYS']
dataSet = [
    [
        item['BWID'],
        item['BWNAME'],
        item['LATITUDE'],
        item['LONGITUDE'],
        item['BWKEY'],
    ]
    for item in data
]
dataSetDF = pd.DataFrame(dataSet, columns=["id", "name", "long", "lat", "place"])
geometry = [Point(x, y) for x, y in zip(dataSetDF['long'], dataSetDF['lat'])]

In [ ]:
gdf = GeoDataFrame(dataSetDF, geometry=geometry).set_index('id')

In [ ]:
gdf = gdf.set_crs(4258)

In [ ]:
gdf.crs

<Geographic 2D CRS: EPSG:4258>
Name: ETRS89
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Moldova; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain; Sweden; Switzerland; United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-16.1, 32.88, 40.18, 84.73)
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
gdf.to_crs(4326)

,name,long,lat,place,geometry
id,,,,,
BEVL_VMM_10,De Panne - Leopold,51.1020,2.5772,De Panne,POINT (51.10200 2.57720)
BEVL_VMM_100,Koksijde - Duinpark - Oostduinkerke,51.1385,2.6796,Koksijde,POINT (51.13850 2.67960)
BEVL_VMM_107760,Bocholt - Goolderheide,51.1739,5.5354,Bocholt,POINT (51.17390 5.53540)
BEVL_VMM_110,Koksijde - Groenendijk - Oostduinkerke,51.1448,2.6996,Koksijde,POINT (51.14480 2.69960)
BEVL_VMM_115,Nieuwpoort - Leopoldplein,51.1485,2.7080,Nieuwpoort,POINT (51.14850 2.70800)
...,...,...,...,...,...
BEVL_VMM_848775,Dessel - Campinastrand,51.2630,5.1362,Dessel,POINT (51.26300 5.13620)
BEVL_VMM_848795,Mol - Familiestrand Postel,51.2778,5.1569,Mol,POINT (51.27780 5.15690)
BEVL_VMM_871500,Jabbeke - Klein Strand,51.1860,3.1014,Jabbeke,POINT (51.18600 3.10140)


In [ ]:
gdf.to_csv('vlaanderen.csv')